In [5]:
# !unzip output.zip

unzip:  cannot find or open data/output.zip, data/output.zip.zip or data/output.zip.ZIP.


In [7]:

!pip install spektral

In [1]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [3]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 1  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)

    features = sp.csr_matrix(x).astype('float32')
    # adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    g = nx.DiGraph()
    g.add_nodes_from(graph.keys())
    for k, v in graph.items():
      g.add_edges_from(([(k, t) for t in v]))
      g.add_edges_from([(k, k)])
    adj = nx.adjacency_matrix(g)
    labels = np.array(y).astype('float32').reshape((-1,1))

    print(adj.shape)
    print(features.shape)
    return adj, features, labels
  
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels


class CircuitDataset(Dataset):
    def read(self):
        circuits = []
        circs = ['c6288', 'c17', 'c5315', 'c432', 'c499', 'c880', 'c1355', 'c1908', 'c3540']
        # circs = ['c17']
        for circ in circs:
            A, X, labels = load_data(circ, '../data/output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
            print(f"{circ}: {sum(labels)}, {len(labels)}")
        return circuits

dataset = CircuitDataset(transforms=[LayerPreprocess(GCNConv)])

# Parameters
F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
# [8 6 5 0 1 4 2] [3] [7]
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]
# dataset_tr = dataset[[8,6,5,1,2, 0, 3, 4, 7]]
# dataset_va = dataset[[0,3]]
# dataset_te = dataset[4,7]
# loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
# loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
# loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)



(6288, 6288)
(6288, 4)
c6288: [846.], 6288
(17, 17)
(17, 4)
c17: [2.], 17
(5315, 5315)
(5315, 4)
c5315: [599.], 5315
(432, 432)
(432, 4)
c432: [60.], 432
(499, 499)
(499, 4)
c499: [50.], 499
(880, 880)
(880, 4)
c880: [114.], 880
(1355, 1355)
(1355, 4)
c1355: [192.], 1355
(1908, 1908)
(1908, 4)
c1908: [257.], 1908
(3540, 3540)
(3540, 4)
c3540: [406.], 3540
[7 3 0 2 4 1 6] [8] [5]


In [4]:
# loader_tr = DisjointLoader(dataset_tr, batch_size=dataset_tr.n_graphs, epochs=epochs, node_level=True)
# batch = loader_tr.__next__()
# inputs, target = batch
# x, a, i = inputs
# print(x.shape, a.shape, target.shape)
# print(x.dtype, a.dtype, target.dtype)
# sum([ g.n_nodes for g in dataset_tr ])

In [5]:
dataset[3]

Graph(n_nodes=432, n_node_features=4, n_edge_features=None, n_labels=1)

In [6]:
N = sum([ g.n_nodes for g in dataset.graphs ])
n_out = 1
N

20234

In [7]:
channels = 50          # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 1000            # Number of training epochs
es_patience = 10        # Patience for early stopping
# N = 

# Model definition
X_in = Input(shape=(F, ))
print("X_in: ", X_in.shape)
fltr_in = Input((None, ), sparse=True)
# mask_in = Input((None, ))

dropout_1 = Dropout(dropout)(X_in)
print("drp 1", dropout_1)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])
print("graph conv 1", graph_conv_1)
dropout_2 = Dropout(dropout)(graph_conv_1)
print("drp 2", dropout_2)
graph_conv_2 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_2, fltr_in])
dropout_3 = Dropout(dropout)(graph_conv_1)
print("drp 2", dropout_2)
graph_conv_3 = GCNConv(n_out,
                         activation='sigmoid',
                         use_bias=False)([dropout_3, fltr_in])
print("graph conv 2", graph_conv_2)
# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_3)
optimizer = Adam(lr=learning_rate)
# model.compile(optimizer=optimizer,
#               loss='categorical_crossentropy',
#               weighted_metrics=['acc'])

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./DetectFaultInCircuit',
)
callback_GCN = [tbCallBack_GCN]

X_in:  (None, 4)
drp 1 KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='dropout/Identity:0', description="created by layer 'dropout'")
graph conv 1 KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='gcn_conv/Relu:0', description="created by layer 'gcn_conv'")
drp 2 KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='dropout_1/Identity:0', description="created by layer 'dropout_1'")
drp 2 KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='dropout_1/Identity:0', description="created by layer 'dropout_1'")
graph conv 2 KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='gcn_conv_1/Relu:0', description="created by layer 'gcn_conv_1'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Con

In [8]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.layers import GCNConv, GlobalSumPool
from spektral.layers.ops import sp_matrix_to_sp_tensor

In [9]:
optimizer = Adam()
# loss_fn = SparseCategoricalCrossentropy()
# loss_fn = BinaryCrossentropy()
loss_fn = binary_crossentropy

# def input_mask(target):
#     minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
#     zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])[:minimum]
#     ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])[:minimum]
    
#     stacked_inp = tf.reshape(tf.stack([zeroes_index, ones_index]), (-1,1))
#     zeros_stack = tf.reshape(tf.stack([stacked_inp, tf.zeros((minimum*2,1), dtype='int64')], axis=1), (-1,2))
#     sparsed_data = tf.SparseTensor(zeros_stack, tf.ones((minimum*2, )), target.shape)
#     return tf.sparse.to_dense(tf.sparse.reorder(sparsed_data), default_value = 0. )

def masked_loss_and_accuracy(target, predictions):
#     values_keep = tf.gather( logits[0], idx_keep )
    loss = loss_fn(target, predictions)
    minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
    zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])[:minimum]
    ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])[:minimum]
    loss_zeroes =  tf.reduce_sum(tf.gather(loss, zeroes_index))
    loss_ones = tf.reduce_sum(tf.gather(loss, ones_index))
    loss = loss_zeroes + loss_ones
#     loss = tf.reduce_mean(tf.multiply(loss, mask))
    bin_acc = binary_accuracy(target, predictions)
    acc = tf.reduce_mean(bin_acc)
    return loss, acc

# Training function
@tf.function
def train_on_batch(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs[:-1], training=True)
        loss, acc = masked_loss_and_accuracy(target, predictions)
        loss = loss + sum(model.losses)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc


# Evaluation function
def evaluate(loader):
    step = 0
    results = []
    for batch in loader:
        step += 1
        inputs, target = batch
        predictions = model(inputs[:-1], training=False)
        loss, acc = masked_loss_and_accuracy(target, predictions)
        results.append((loss, acc, len(target)))  # Keep track of batch size
        if step == loader.steps_per_epoch:
            results = np.array(results)
            return np.average(results[:, :-1], 0, weights=results[:, -1])

patience = 10
# Setup training
best_val_loss = 99999
current_patience = patience
step = 0

# Training loop
results_tr = []
for batch in loader_tr:
    step += 1

    # Training step
    inputs, target = batch
    loss, acc = train_on_batch(inputs, target)
    results_tr.append((loss, acc, len(target)))

    if step == loader_tr.steps_per_epoch:
        results_va = evaluate(loader_va)
        if results_va[0] < best_val_loss:
            best_val_loss = results_va[0]
            current_patience = patience
            results_te = evaluate(loader_te)
        else:
            current_patience -= 1
            if current_patience == 0:
                print("Early stopping")
                break

        # Print results
        results_tr = np.array(results_tr)
        results_tr = np.average(results_tr[:, :-1], 0, weights=results_tr[:, -1])
        print(
            "Train loss: {:.4f}, acc: {:.4f} | "
            "Valid loss: {:.4f}, acc: {:.4f} | "
            "Test loss: {:.4f}, acc: {:.4f}".format(
                *results_tr, *results_va, *results_te
            )
        )

        # Reset epoch
        results_tr = []
        step = 0

Train loss: 43913.8150, acc: 0.7521 | Valid loss: 17388.7012, acc: 0.8853 | Test loss: 566.0920, acc: 0.8705
Train loss: 37155.0580, acc: 0.7086 | Valid loss: 13034.7754, acc: 0.8740 | Test loss: 425.3871, acc: 0.8648
Train loss: 33909.9789, acc: 0.6604 | Valid loss: 8666.3271, acc: 0.8359 | Test loss: 308.1677, acc: 0.8182
Train loss: 28042.5621, acc: 0.6229 | Valid loss: 5070.2036, acc: 0.7703 | Test loss: 220.7516, acc: 0.6250
Train loss: 29035.9724, acc: 0.5716 | Valid loss: 2239.0630, acc: 0.5116 | Test loss: 182.3976, acc: 0.4443
Train loss: 24738.4734, acc: 0.5101 | Valid loss: 1579.2344, acc: 0.1441 | Test loss: 168.1138, acc: 0.3023
Train loss: 23626.3851, acc: 0.4874 | Valid loss: 1623.7969, acc: 0.1263 | Test loss: 168.1138, acc: 0.3023
Train loss: 25089.6869, acc: 0.4647 | Valid loss: 2049.3562, acc: 0.1206 | Test loss: 168.1138, acc: 0.3023
Train loss: 23842.4135, acc: 0.4454 | Valid loss: 2277.2185, acc: 0.1186 | Test loss: 168.1138, acc: 0.3023
Train loss: 24110.8251, ac

In [20]:
# Evaluate model
from sklearn.metrics import classification_report

X_te = dataset[4].x
A_te = dataset[4].a
y_te = dataset[4].y

y_pred = model.predict([X_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, y_pred)
print('GCN Classification Report: \n {}'.format(report))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [19]:
(y_pred >= 0.5).astype(int).sum()

355

In [146]:
acc = tf.keras.metrics.binary_accuracy(y_te, y_pred).numpy()
minimum = np.array([(y_te == 0).sum(), (y_te == 1).sum()]).min()
zeroes_index = np.random.choice(np.where(y_te == 0)[0], minimum, replace=False)
ones_index = np.random.choice(np.where(y_te == 1)[0], minimum, replace=False)
mask = np.zeros_like(y_te)
mask[zeroes_index] = 1
mask[ones_index] = 1
# np.multiply(bin_acc, mask).sum()


In [231]:
target = tf.convert_to_tensor(y_te)
minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])[:minimum]
ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])[:minimum]
mask = tf.Variable(tf.zeros_like(target))
stacked_inp = tf.reshape(tf.stack([zeroes_index, ones_index]), (-1,1))
zeros_stack = tf.reshape(tf.stack([stacked_inp, tf.zeros((minimum*2,1), dtype='int64')], axis=1), (-1,2))

In [208]:
target = tf.convert_to_tensor(y_te)
minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
zeroes_index = tf.random.shuffle(tf.where(target == 0))[:minimum, :]
ones_index = tf.random.shuffle(tf.where(target == 1))[:minimum, :]
mask = tf.Variable(tf.zeros_like(target))
stacked_inp = tf.stack([zeroes_index, ones_index], axis=1)

In [232]:
# tf.SparseTensor(stacked_inp, tf.ones((minimum*2, )), target.shape)
zeroes_index.shape, ones_index.shape, stacked_inp.shape, tf.ones((minimum*2, )).shape, target.shape, zeros_stack.shape

(TensorShape([50]),
 TensorShape([50]),
 TensorShape([100, 1]),
 TensorShape([100]),
 TensorShape([499, 1]),
 TensorShape([100, 2]))

In [237]:
sparsed_data = tf.SparseTensor(zeros_stack, tf.ones((minimum*2, )), target.shape)
mask = tf.sparse.to_dense(tf.sparse.reorder(sparsed_data), default_value = 0. )

In [240]:
tf.reduce_sum(densed)

<tf.Tensor: shape=(), dtype=float32, numpy=100.0>

In [200]:
def top_k_filtering( logits, top_k = 5):
    
    # a[...,1] equivalent to a[: ,: ,1 ]
    indices_to_remove = logits < tf.math.top_k(logits,top_k)[0][..., -1, None]
    # indices_to_remove is a tensor of bool values e.g. [ True, False, False, ..., True ]

    # 1d indices
    idx_remove = tf.where( indices_to_remove == 0 )[:,-1]
    idx_keep = tf.where( indices_to_remove == 1 )[:,-1]
    
    values_remove = tf.tile( [-float('inf')], [tf.shape(idx_remove)[0]] ) 
    values_keep = tf.gather( logits[0], idx_keep )

    # to create a sparse vector we still need 2d indices like [ [0,1], [0,2], [0,10] ]
    # create vectors of 0's that we'll later stack with the actual indices
    zeros_remove = tf.zeros_like(idx_remove)
    zeros_keep = tf.zeros_like(idx_keep)

    idx_remove = tf.stack( [ zeros_remove, idx_remove], axis=1 )
    idx_keep = tf.stack( [ zeros_keep, idx_keep], axis=1 )

    # now we can create a sparse matrix
    logits_remove = tf.SparseTensor( idx_remove, values_remove, tf.shape(logits, out_type = tf.int64))
    logits_keep = tf.SparseTensor( idx_keep, values_keep, tf.shape(logits, out_type = tf.int64))

    # add together the two matrices (need to convert them to dense first)
    filtered_logits = tf.add(
        tf.sparse.to_dense(logits_remove, default_value = 0. ),
        tf.sparse.to_dense(logits_keep, default_value = 0. )
    )

    return filtered_logits

In [255]:
tf.sequence_mask([1, 3, 2], target.shape[0])

<tf.Tensor: shape=(3, 432), dtype=bool, numpy=
array([[ True, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True, False, ..., False, False, False]])>

In [67]:
# from sklearn.manifold import TSNE
# layer_outputs = [layer.output for layer in model.layers]
# activation_model = Model(inputs=model.input, outputs=layer_outputs)
# activations = activation_model.predict([X_te, A_te],batch_size=X_te.shape[0])

# #Get t-SNE Representation
# #get the hidden layer representation after the first GCN layer
# x_tsne = TSNE(n_components=2).fit_transform(activations[3]) 

In [202]:
# import matplotlib.pyplot as plt
# def plot_tSNE(labels_encoded,x_tsne):
#     color_map = np.argmax(labels_encoded, axis=1)
#     plt.figure(figsize=(10,10))
#     for cl in range(1):
#         indices = np.where(color_map==cl)
#         indices = indices[0]
#         plt.scatter(x_tsne[indices,0], x_tsne[indices, 1], label=cl)
#     plt.legend()
#     plt.show()
    
# plot_tSNE(y_te,x_tsne)

In [13]:
# y_true = [[1], [1], [0], [0]]
# y_pred = [[1], [1], [0], [0]]
tf.keras.metrics.binary_accuracy(y_te, y_pred).numpy()



array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1.

In [12]:
(y_te == 0).sum(), (y_te == 1).sum() 

(449, 50)

355

In [226]:
y_true = [[1], [1], [0], [0]]
y_pred = [[0.8], [0.8], [0.3], [0.3]]
m = tf.keras.metrics.binary_accuracy(y_true, y_pred)



In [45]:
y_pred

array([[0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36716142],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.36763102],
       [0.36708796],
       [0.367